# Edmonton Metro line project 

## By Jayden Geisler

#### I live in Edmonton Alberta Canada and if you live here too you will know that the LRT system (Light Rail Transit) is not very good for people who live on the east and west side of the city. In this project I attempt to use data of the population and the city to try and plot out the most efficient new LRT line. A new efficient line would value the number of people it reaches and if they will use it as well as the value of the destinations it leads to.

#### I will do this by making 2 data sets. One will be the population of the neighborhoods along with income and education percentage which was attained form the city of Edmonton. The other data set will be venues visited within the neighborhood; this will be obtained through foursquare API. 

#### To solve the problem, I will do two k means on each of the 2 data set. On the venue data set there will be between 3 to 5 clusters and the highest valued cluster will be used as “endpoints” for the new line. For the second data set there will have more clusters, between 8-12 so there are many different values for each neighborhood. Reading a paper for 2017 from the American Public Transportation Association 51% of the people who ride transit hold a bachelor’s degree, for income low - middle income people uses transit the most. After the 2 clusters have been mapped out the center points of the data will be connected through the use of a A-star algorithm that uses both cluster values as a heuristic. 

#### American Public Transportation Association article: https://www.apta.com/wp-content/uploads/Resources/resources/reportsandpublications/Documents/APTA-Who-Rides-Public-Transportation-2017.pdf 


In [1]:
import pandas as pd
import numpy as np



from geopy.geocoders import Nominatim


import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 

# Download and Clean up the data set 

Data has been found from city of edmonton website 

neighbourhood location url : https://data.edmonton.ca/City-Administration/City-of-Edmonton-Neighbourhoods-Centroid-Point-/3b6m-fezs

neighbourhood population url : https://data.edmonton.ca/Census/2012-Census-Population-By-Citizenship-Neighbourhoo/nc6t-tngg

neighbourhood income url : https://data.edmonton.ca/Census/2016-Census-Population-by-Household-Income-Neighbo/jkjx-2hix

neighbourhood education url:  https://data.edmonton.ca/Census/2016-Census-Population-by-Highest-Educational-Atta/f7ms-98xd

In [2]:
url = 'C:\Projects\Coursera_Capstone/City_of_Edmonton_-_Neighbourhoods__Centroid_Point_.csv'
df = pd.read_csv(url)
df.head()

,Neighbourhood Number,Neighbourhood Name,Area Sq Km,Latitude,Longitude,Location
0,8886,Edmonton South East,20.424907,53.381222,-113.369114,"(53.38122208627588, -113.3691136163807)"
1,4670,Winterburn Industrial Area West,4.340207,53.562804,-113.701391,"(53.562803614238916, -113.70139052232909)"
2,4487,Secord,2.643762,53.533937,-113.701353,"(53.53393710914364, -113.7013527663504)"
3,4620,Westview Village,0.925300,53.552222,-113.696668,"(53.552222069483065, -113.69666819890175)"
4,6664,Decoteau North,3.750301,53.433217,-113.350674,"(53.43321724004238, -113.350674197389)"


In [3]:
column_names = ['Neighbourhood Number','Neighbourhood','Latitude','Longitude']

edm_hood = pd.DataFrame(columns = column_names)

edm_hood['Neighbourhood Number'] = df['Neighbourhood Number']
edm_hood['Neighbourhood'] = df['Neighbourhood Name']
edm_hood['Latitude'] = df['Latitude']
edm_hood['Longitude'] = df['Longitude']

In [4]:
edm_hood.head()

,Neighbourhood Number,Neighbourhood,Latitude,Longitude
0,8886,Edmonton South East,53.381222,-113.369114
1,4670,Winterburn Industrial Area West,53.562804,-113.701391
2,4487,Secord,53.533937,-113.701353
3,4620,Westview Village,53.552222,-113.696668
4,6664,Decoteau North,53.433217,-113.350674


In [5]:
url = 'C:\Projects\Coursera_Capstone/2012_Census_-_Population_By_Citizenship__Neighbourhood_.csv'
edm_hood_pop = pd.read_csv(url)
edm_hood_pop.head()

,WARD,NEIGHBOURHOOD_NUMBER,NEIGHBOURHOOD NAME,CANADIAN_CITIZEN,NON-CANADIAN_CITIZEN,NO_RESPONSE
0,Ward 1,4010,ALBERTA PARK INDUSTRIAL,0,0,0
1,Ward 1,4020,ALDERGROVE,4756,570,189
2,Ward 1,4011,ANTHONY HENDAY,0,0,0
3,Ward 1,4030,ARMSTRONG INDUSTRIAL,0,0,125
4,Ward 1,4040,BELMEAD,3942,480,181


In [6]:
url = 'C:\Projects\Coursera_Capstone/2016_Census_-_Population_by_Household_Income__Neighbourhood_Ward_.csv'
edm_hood_income = pd.read_csv(url)
edm_hood_income.head()

,Ward,Neighbourhood Number,Neighbourhood Name,"Less than $30,000","$30,000 to less than $60,000","$60,000 to less than $100,000","$100,000 to less than $125,000","$125,000 to less than $150,000","$150,000 to less than $200,000","$200,000 to less than $250,000","$250,000 or more",No Response
0,WARD 1,3140,CRESTWOOD,56,91,90,52,26,58,36,103,404
1,WARD 1,3330,PARKVIEW,51,116,149,93,65,94,60,95,577
2,WARD 5,4220,JAMIESON PLACE,26,71,103,78,64,65,17,13,882
3,WARD 9,5454,RUTHERFORD,130,368,621,334,255,273,99,77,1938
4,WARD 3,2461,CRYSTALLINA NERA EAST,0,0,0,0,0,0,0,0,0


In [7]:
url = 'C:\Projects\Coursera_Capstone/2016_Census_-_Population_by_Highest_Educational_Attainment__Neighbourhood_Ward_.csv'
edm_hood_edu = pd.read_csv(url)
edm_hood_edu.head()

,Ward,Neighbourhood Number,Neighbourhood Name,"No Certificate, Diploma or Degree",High School Diploma or Equivalent,Trades Certificate,Registered Apprenticeship Certificate,College Certificate or Diploma,University Certificate Below Bachelor's Level,Bachelor's Degree,University Certificate or Diploma above Bachelor Level,Medical Degree,Master's Degree,Earned Doctorate,No Response
0,WARD 1,3140,CRESTWOOD,63,280,36,6,256,60,415,61,38,137,42,25
1,WARD 1,3330,PARKVIEW,55,445,75,14,257,67,552,137,58,194,64,141
2,WARD 4,2690,RURAL NORTH EAST SOUTH STURGEON,14,126,24,3,64,14,38,4,1,4,2,42
3,WARD 5,4720,GLASTONBURY,105,692,156,23,624,89,581,75,9,125,25,2218
4,WARD 6,3381,RIVER VALLEY GLENORA,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#clean up population dataframe where no citizen data is given 
index = 0
for i in edm_hood_pop['CANADIAN_CITIZEN']:
   
    if i == 0 & edm_hood_pop['NON-CANADIAN_CITIZEN'][index] == 0 & edm_hood_pop['NO_RESPONSE'][index] == 0:    
        edm_hood_pop.drop([index],inplace= True)
        
    index += 1

In [8]:
#clean the population dataframe
edm_hood_pop.rename(columns = {'NEIGHBOURHOOD_NUMBER': 'Neighbourhood Number', 'NEIGHBOURHOOD NAME': 'Neighbourhood'}, inplace = True)
edm_hood_pop['Population'] = edm_hood_pop[['CANADIAN_CITIZEN','NON-CANADIAN_CITIZEN','NO_RESPONSE']].sum(axis=1)
edm_hood_pop.drop(columns = ['WARD','CANADIAN_CITIZEN','NON-CANADIAN_CITIZEN','NO_RESPONSE'],inplace = True)

In [9]:
edm_hood_pop.reset_index()

,index,Neighbourhood Number,Neighbourhood,Population
0,0,4010,ALBERTA PARK INDUSTRIAL,0
1,1,4020,ALDERGROVE,5515
2,2,4011,ANTHONY HENDAY,0
3,3,4030,ARMSTRONG INDUSTRIAL,125
4,4,4040,BELMEAD,4603
...,...,...,...,...
370,370,6443,TAMARACK,2348
371,371,6216,THE ORCHARDS AT ELLERSLIE,0
372,372,6662,WALKER,2036
373,373,6770,WEINLOS,3445


In [10]:
#clan up income data if no data given of income data set 
index = 0
for i in edm_hood_income['Less than $30,000']:
   
    if i == 0 & edm_hood_income['$60,000 to less than $100,000'][index] == 0 & edm_hood_income['$200,000 to less than $250,000'][index] == 0:    
        edm_hood_income.drop([index],inplace= True)
        
    index += 1

In [11]:
edm_hood_income.reset_index()

,index,Ward,Neighbourhood Number,Neighbourhood Name,"Less than $30,000","$30,000 to less than $60,000","$60,000 to less than $100,000","$100,000 to less than $125,000","$125,000 to less than $150,000","$150,000 to less than $200,000","$200,000 to less than $250,000","$250,000 or more",No Response
0,0,WARD 1,3140,CRESTWOOD,56,91,90,52,26,58,36,103,404
1,1,WARD 1,3330,PARKVIEW,51,116,149,93,65,94,60,95,577
2,2,WARD 5,4220,JAMIESON PLACE,26,71,103,78,64,65,17,13,882
3,3,WARD 9,5454,RUTHERFORD,130,368,621,334,255,273,99,77,1938
4,5,WARD 11,6470,MEYONOHK,47,138,156,62,23,28,8,0,650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,381,WARD 10,5430,ROYAL GARDENS,121,199,224,68,31,38,19,7,695
264,382,WARD 10,5180,ERMINESKIN,367,531,322,94,60,45,28,11,986
265,383,WARD 7,2040,BEACON HEIGHTS,142,152,190,62,33,24,6,3,591
266,385,WARD 2,3480,HUDSON,21,98,163,63,32,31,4,15,373


In [12]:
# make less columns, combinne and make the numbers a percentage 
cols = edm_hood_income.columns
colslist = list(cols)
print(colslist[3:])
edm_hood_income['total'] = edm_hood_income[colslist[3:]].sum(axis=1)
edm_hood_income


['Less than $30,000', '$30,000 to less than $60,000', '$60,000 to less than $100,000', '$100,000 to less than $125,000', '$125,000 to less than $150,000', '$150,000 to less than $200,000', '$200,000 to less than $250,000', '$250,000 or more', 'No Response']


,Ward,Neighbourhood Number,Neighbourhood Name,"Less than $30,000","$30,000 to less than $60,000","$60,000 to less than $100,000","$100,000 to less than $125,000","$125,000 to less than $150,000","$150,000 to less than $200,000","$200,000 to less than $250,000","$250,000 or more",No Response,total
0,WARD 1,3140,CRESTWOOD,56,91,90,52,26,58,36,103,404,916
1,WARD 1,3330,PARKVIEW,51,116,149,93,65,94,60,95,577,1300
2,WARD 5,4220,JAMIESON PLACE,26,71,103,78,64,65,17,13,882,1319
3,WARD 9,5454,RUTHERFORD,130,368,621,334,255,273,99,77,1938,4095
5,WARD 11,6470,MEYONOHK,47,138,156,62,23,28,8,0,650,1112
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,WARD 10,5430,ROYAL GARDENS,121,199,224,68,31,38,19,7,695,1402
382,WARD 10,5180,ERMINESKIN,367,531,322,94,60,45,28,11,986,2444
383,WARD 7,2040,BEACON HEIGHTS,142,152,190,62,33,24,6,3,591,1203
385,WARD 2,3480,HUDSON,21,98,163,63,32,31,4,15,373,800


In [13]:
edm_hood_income['Less than $60,000'] = edm_hood_income[['Less than $30,000','$30,000 to less than $60,000']].sum(axis=1)/edm_hood_income['total']
edm_hood_income['$60,000 to $125,000'] = edm_hood_income[['$60,000 to less than $100,000','$100,000 to less than $125,000']].sum(axis=1)/edm_hood_income['total']
edm_hood_income['$125,000 to $200,000'] = edm_hood_income[['$125,000 to less than $150,000','$150,000 to less than $200,000']].sum(axis=1)/edm_hood_income['total']
edm_hood_income['$200,000 or more'] = edm_hood_income[['$200,000 to less than $250,000','$250,000 or more']].sum(axis=1)/edm_hood_income['total']
edm_hood_income['No Response income'] = edm_hood_income['No Response']/edm_hood_income['total']
edm_hood_income

,Ward,Neighbourhood Number,Neighbourhood Name,"Less than $30,000","$30,000 to less than $60,000","$60,000 to less than $100,000","$100,000 to less than $125,000","$125,000 to less than $150,000","$150,000 to less than $200,000","$200,000 to less than $250,000","$250,000 or more",No Response,total,"Less than $60,000","$60,000 to $125,000","$125,000 to $200,000","$200,000 or more",No Response income
0,WARD 1,3140,CRESTWOOD,56,91,90,52,26,58,36,103,404,916,0.160480,0.155022,0.091703,0.151747,0.441048
1,WARD 1,3330,PARKVIEW,51,116,149,93,65,94,60,95,577,1300,0.128462,0.186154,0.122308,0.119231,0.443846
2,WARD 5,4220,JAMIESON PLACE,26,71,103,78,64,65,17,13,882,1319,0.073541,0.137225,0.097801,0.022745,0.668688
3,WARD 9,5454,RUTHERFORD,130,368,621,334,255,273,99,77,1938,4095,0.121612,0.233211,0.128938,0.042979,0.473260
5,WARD 11,6470,MEYONOHK,47,138,156,62,23,28,8,0,650,1112,0.166367,0.196043,0.045863,0.007194,0.584532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,WARD 10,5430,ROYAL GARDENS,121,199,224,68,31,38,19,7,695,1402,0.228245,0.208274,0.049215,0.018545,0.495720
382,WARD 10,5180,ERMINESKIN,367,531,322,94,60,45,28,11,986,2444,0.367430,0.170213,0.042962,0.015957,0.403437
383,WARD 7,2040,BEACON HEIGHTS,142,152,190,62,33,24,6,3,591,1203,0.244389,0.209476,0.047382,0.007481,0.491272
385,WARD 2,3480,HUDSON,21,98,163,63,32,31,4,15,373,800,0.148750,0.282500,0.078750,0.023750,0.466250


In [14]:
colstodrop = ['Ward','Less than $30,000','$30,000 to less than $60,000', '$60,000 to less than $100,000', '$100,000 to less than $125,000', '$125,000 to less than $150,000', '$150,000 to less than $200,000', '$200,000 to less than $250,000', '$250,000 or more','total','No Response']
edm_hood_income.drop(colstodrop, axis = 1,inplace = True)

In [15]:
edm_hood_income.reset_index()

,index,Neighbourhood Number,Neighbourhood Name,"Less than $60,000","$60,000 to $125,000","$125,000 to $200,000","$200,000 or more",No Response income
0,0,3140,CRESTWOOD,0.160480,0.155022,0.091703,0.151747,0.441048
1,1,3330,PARKVIEW,0.128462,0.186154,0.122308,0.119231,0.443846
2,2,4220,JAMIESON PLACE,0.073541,0.137225,0.097801,0.022745,0.668688
3,3,5454,RUTHERFORD,0.121612,0.233211,0.128938,0.042979,0.473260
4,5,6470,MEYONOHK,0.166367,0.196043,0.045863,0.007194,0.584532
...,...,...,...,...,...,...,...,...
263,381,5430,ROYAL GARDENS,0.228245,0.208274,0.049215,0.018545,0.495720
264,382,5180,ERMINESKIN,0.367430,0.170213,0.042962,0.015957,0.403437
265,383,2040,BEACON HEIGHTS,0.244389,0.209476,0.047382,0.007481,0.491272
266,385,3480,HUDSON,0.148750,0.282500,0.078750,0.023750,0.466250


In [17]:
#clean up education if no data is given education data set 
index = 0
for i in edm_hood_edu['No Certificate, Diploma or Degree']:
   
    if i == 0 & edm_hood_edu['High School Diploma or Equivalent'][index] == 0 & edm_hood_edu["Bachelor\'s Degree"][index] == 0:    
        edm_hood_edu.drop([index],inplace= True)
        
    index += 1

In [21]:
edm_hood_edu.reset_index()

,index,Ward,Neighbourhood Number,Neighbourhood Name,"No Certificate, Diploma or Degree",High School Diploma or Equivalent,Trades Certificate,Registered Apprenticeship Certificate,College Certificate or Diploma,University Certificate Below Bachelor's Level,...,Medical Degree,Master's Degree,Earned Doctorate,No Response,total,High School Diploma or below,"Trades, College Certificate or Diploma",University Degree,"Medical degree, Master's or Doctorate",No Response education
0,0,WARD 1,3140,CRESTWOOD,63,280,36,6,256,60,...,38,137,42,25,1419,0.241720,0.252290,0.335447,0.152925,0.017618
1,1,WARD 1,3330,PARKVIEW,55,445,75,14,257,67,...,58,194,64,141,2059,0.242836,0.200583,0.334628,0.153473,0.068480
2,2,WARD 4,2690,RURAL NORTH EAST SOUTH STURGEON,14,126,24,3,64,14,...,1,4,2,42,336,0.416667,0.312500,0.125000,0.020833,0.125000
3,3,WARD 5,4720,GLASTONBURY,105,692,156,23,624,89,...,9,125,25,2218,4722,0.168784,0.188903,0.138924,0.033672,0.469716
4,6,WARD 1,4060,BRITANNIA YOUNGSTOWN,261,658,119,9,343,56,...,1,46,0,843,2568,0.357866,0.205218,0.090343,0.018302,0.328271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,378,WARD 3,3180,DUNLUCE,225,869,216,17,538,102,...,3,41,7,2008,4357,0.251090,0.200367,0.075970,0.011705,0.460868
270,380,WARD 3,2511,MAYLIEWAN,135,523,130,7,286,117,...,8,30,8,1272,2708,0.242984,0.199409,0.070901,0.016987,0.469719
271,381,WARD 2,3250,KENSINGTON,214,485,106,12,241,29,...,0,26,2,1060,2323,0.300904,0.167025,0.063711,0.012053,0.456307
272,384,WARD 4,2710,SIFTON PARK,49,113,26,6,72,25,...,2,10,1,482,826,0.196126,0.156174,0.048426,0.015738,0.583535


In [16]:
# make less columns, combinne and make the numbers a percentage 
cols = edm_hood_edu.columns
colslist = list(cols)
print(colslist[3:])
edm_hood_edu['total'] = edm_hood_edu[colslist[3:]].sum(axis=1)
edm_hood_edu

['No Certificate, Diploma or Degree', 'High School Diploma or Equivalent', 'Trades Certificate', 'Registered Apprenticeship Certificate', 'College Certificate or Diploma', "University Certificate Below Bachelor's Level", "Bachelor's Degree", 'University Certificate or Diploma above Bachelor Level', 'Medical Degree', "Master's Degree", 'Earned Doctorate', 'No Response']


,Ward,Neighbourhood Number,Neighbourhood Name,"No Certificate, Diploma or Degree",High School Diploma or Equivalent,Trades Certificate,Registered Apprenticeship Certificate,College Certificate or Diploma,University Certificate Below Bachelor's Level,Bachelor's Degree,University Certificate or Diploma above Bachelor Level,Medical Degree,Master's Degree,Earned Doctorate,No Response,total
0,WARD 1,3140,CRESTWOOD,63,280,36,6,256,60,415,61,38,137,42,25,1419
1,WARD 1,3330,PARKVIEW,55,445,75,14,257,67,552,137,58,194,64,141,2059
2,WARD 4,2690,RURAL NORTH EAST SOUTH STURGEON,14,126,24,3,64,14,38,4,1,4,2,42,336
3,WARD 5,4720,GLASTONBURY,105,692,156,23,624,89,581,75,9,125,25,2218,4722
4,WARD 6,3381,RIVER VALLEY GLENORA,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,WARD 8,6620,RIVER VALLEY GOLD BAR,0,0,0,0,0,0,0,0,0,0,0,0,0
384,WARD 4,2710,SIFTON PARK,49,113,26,6,72,25,29,11,2,10,1,482,826
385,WARD 4,2080,BELVEDERE,272,656,145,23,351,40,180,20,0,42,2,1696,3427
386,WARD 1,4100,CARLETON SQUARE INDUSTRIAL,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
#combine a few columns and make them a percentage 
edm_hood_edu['High School Diploma or below'] = edm_hood_edu[['High School Diploma or Equivalent','No Certificate, Diploma or Degree']].sum(axis=1)/edm_hood_edu['total']

edm_hood_edu['Trades, College Certificate or Diploma'] = edm_hood_edu[['Trades Certificate', 'Registered Apprenticeship Certificate', 'College Certificate or Diploma', "University Certificate Below Bachelor's Level"]].sum(axis=1)/edm_hood_edu['total']

edm_hood_edu['University Degree'] = edm_hood_edu[["Bachelor's Degree", 'University Certificate or Diploma above Bachelor Level']].sum(axis=1)/edm_hood_edu['total']

edm_hood_edu['Medical degree, Master\'s or Doctorate'] = edm_hood_edu[['Medical Degree', "Master's Degree", 'Earned Doctorate']].sum(axis=1)/edm_hood_edu['total']

edm_hood_edu['No Response education'] = edm_hood_edu['No Response']/edm_hood_edu['total']

edm_hood_edu

,Ward,Neighbourhood Number,Neighbourhood Name,"No Certificate, Diploma or Degree",High School Diploma or Equivalent,Trades Certificate,Registered Apprenticeship Certificate,College Certificate or Diploma,University Certificate Below Bachelor's Level,Bachelor's Degree,...,Medical Degree,Master's Degree,Earned Doctorate,No Response,total,High School Diploma or below,"Trades, College Certificate or Diploma",University Degree,"Medical degree, Master's or Doctorate",No Response education
0,WARD 1,3140,CRESTWOOD,63,280,36,6,256,60,415,...,38,137,42,25,1419,0.241720,0.252290,0.335447,0.152925,0.017618
1,WARD 1,3330,PARKVIEW,55,445,75,14,257,67,552,...,58,194,64,141,2059,0.242836,0.200583,0.334628,0.153473,0.068480
2,WARD 4,2690,RURAL NORTH EAST SOUTH STURGEON,14,126,24,3,64,14,38,...,1,4,2,42,336,0.416667,0.312500,0.125000,0.020833,0.125000
3,WARD 5,4720,GLASTONBURY,105,692,156,23,624,89,581,...,9,125,25,2218,4722,0.168784,0.188903,0.138924,0.033672,0.469716
6,WARD 1,4060,BRITANNIA YOUNGSTOWN,261,658,119,9,343,56,210,...,1,46,0,843,2568,0.357866,0.205218,0.090343,0.018302,0.328271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,WARD 3,3180,DUNLUCE,225,869,216,17,538,102,300,...,3,41,7,2008,4357,0.251090,0.200367,0.075970,0.011705,0.460868
380,WARD 3,2511,MAYLIEWAN,135,523,130,7,286,117,142,...,8,30,8,1272,2708,0.242984,0.199409,0.070901,0.016987,0.469719
381,WARD 2,3250,KENSINGTON,214,485,106,12,241,29,132,...,0,26,2,1060,2323,0.300904,0.167025,0.063711,0.012053,0.456307
384,WARD 4,2710,SIFTON PARK,49,113,26,6,72,25,29,...,2,10,1,482,826,0.196126,0.156174,0.048426,0.015738,0.583535


In [22]:
colstodrop = ['Ward','No Certificate, Diploma or Degree', 'High School Diploma or Equivalent', 'Trades Certificate', 'Registered Apprenticeship Certificate', 'College Certificate or Diploma', "University Certificate Below Bachelor's Level", "Bachelor's Degree", 'University Certificate or Diploma above Bachelor Level', 'Medical Degree', "Master's Degree", 'Earned Doctorate', 'total','No Response']
edm_hood_edu.drop(colstodrop, axis = 1,inplace = True)

In [18]:
edm_hood_edu.reset_index()

,index,Ward,Neighbourhood Number,Neighbourhood Name,"No Certificate, Diploma or Degree",High School Diploma or Equivalent,Trades Certificate,Registered Apprenticeship Certificate,College Certificate or Diploma,University Certificate Below Bachelor's Level,Bachelor's Degree,University Certificate or Diploma above Bachelor Level,Medical Degree,Master's Degree,Earned Doctorate,No Response,total
0,0,WARD 1,3140,CRESTWOOD,63,280,36,6,256,60,415,61,38,137,42,25,1419
1,1,WARD 1,3330,PARKVIEW,55,445,75,14,257,67,552,137,58,194,64,141,2059
2,2,WARD 4,2690,RURAL NORTH EAST SOUTH STURGEON,14,126,24,3,64,14,38,4,1,4,2,42,336
3,3,WARD 5,4720,GLASTONBURY,105,692,156,23,624,89,581,75,9,125,25,2218,4722
4,6,WARD 1,4060,BRITANNIA YOUNGSTOWN,261,658,119,9,343,56,210,22,1,46,0,843,2568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,378,WARD 3,3180,DUNLUCE,225,869,216,17,538,102,300,31,3,41,7,2008,4357
270,380,WARD 3,2511,MAYLIEWAN,135,523,130,7,286,117,142,50,8,30,8,1272,2708
271,381,WARD 2,3250,KENSINGTON,214,485,106,12,241,29,132,16,0,26,2,1060,2323
272,384,WARD 4,2710,SIFTON PARK,49,113,26,6,72,25,29,11,2,10,1,482,826


In [20]:
#missing neighbourhoods 
print(edm_hood.shape)
print(edm_hood_pop.shape)
print(edm_hood_edu.shape)
print(edm_hood_income.shape)

(400, 4)
(375, 3)
(274, 21)
(268, 7)


In [23]:
edm_hood

,Neighbourhood Number,Neighbourhood,Latitude,Longitude
0,8886,Edmonton South East,53.381222,-113.369114
1,4670,Winterburn Industrial Area West,53.562804,-113.701391
2,4487,Secord,53.533937,-113.701353
3,4620,Westview Village,53.552222,-113.696668
4,6664,Decoteau North,53.433217,-113.350674
...,...,...,...,...
395,1270,Yellowhead Corridor East,53.579205,-113.451964
396,3450,Woodcroft,53.564595,-113.558327
397,6680,Satoo,53.446373,-113.458892
398,3050,Brown Industrial,53.582399,-113.568007


In [30]:
edm_hood_pop

,Neighbourhood Number,Neighbourhood,Population
0,4010,ALBERTA PARK INDUSTRIAL,0
1,4020,ALDERGROVE,5515
2,4011,ANTHONY HENDAY,0
3,4030,ARMSTRONG INDUSTRIAL,125
4,4040,BELMEAD,4603
...,...,...,...
370,6443,TAMARACK,2348
371,6216,THE ORCHARDS AT ELLERSLIE,0
372,6662,WALKER,2036
373,6770,WEINLOS,3445


In [31]:
merged_edm_data1 = pd.merge(left = edm_hood, right = edm_hood_pop, left_on = 'Neighbourhood Number', right_on = 'Neighbourhood Number')
merged_edm_data2 = pd.merge(left = edm_hood_edu, right = edm_hood_income, left_on = 'Neighbourhood Number', right_on = 'Neighbourhood Number')

In [32]:
edm_hood_total = pd.merge(left = merged_edm_data1, right = merged_edm_data2, left_on = 'Neighbourhood Number', right_on = 'Neighbourhood Number')

In [33]:
edm_hood_total

,Neighbourhood Number,Neighbourhood_x,Latitude,Longitude,Neighbourhood_y,Population,Neighbourhood Name_x,High School Diploma or below,"Trades, College Certificate or Diploma",University Degree,"Medical degree, Master's or Doctorate",No Response education,Neighbourhood Name_y,"Less than $60,000","$60,000 to $125,000","$125,000 to $200,000","$200,000 or more",No Response income
0,4487,Secord,53.533937,-113.701353,SECORD,914,SECORD,0.332569,0.209174,0.150917,0.037615,0.269725,SECORD,0.323894,0.198230,0.131858,0.030973,0.315044
1,4620,Westview Village,53.552222,-113.696668,WESTVIEW VILLAGE,2242,WESTVIEW VILLAGE,0.638655,0.307773,0.025210,0.004202,0.024160,WESTVIEW VILLAGE,0.197507,0.124640,0.021093,0.000959,0.655801
2,5462,Chappelle Area,53.402917,-113.586845,CHAPPELLE AREA,151,CHAPPELLE AREA,0.054283,0.086359,0.082129,0.027141,0.750088,CHAPPELLE AREA,0.074909,0.263094,0.129111,0.033496,0.499391
3,6216,The Orchards At Ellerslie,53.403087,-113.458166,THE ORCHARDS AT ELLERSLIE,0,THE ORCHARDS AT ELLERSLIE,0.065125,0.105904,0.110164,0.026172,0.692635,THE ORCHARDS AT ELLERSLIE,0.078249,0.327586,0.169761,0.043767,0.380637
4,2670,Rural North East Horse Hill,53.678364,-113.312512,RURAL NORTH EAST HORSE HILL,570,RURAL NORTH EAST HORSE HILL,0.048499,0.013857,0.032333,0.004619,0.900693,RURAL NORTH EAST HORSE HILL,0.056338,0.028169,0.021127,0.028169,0.866197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,3170,Dovercourt,53.575727,-113.559534,DOVERCOURT,2048,DOVERCOURT,0.409021,0.337920,0.152141,0.042049,0.058869,DOVERCOURT,0.255196,0.284065,0.107390,0.024249,0.329099
252,5300,Ogilvie Ridge,53.464904,-113.567069,OGILVIE RIDGE,1001,OGILVIE RIDGE,0.146628,0.170088,0.222874,0.151026,0.309384,OGILVIE RIDGE,0.038889,0.108333,0.086111,0.175000,0.591667
253,3450,Woodcroft,53.564595,-113.558327,WOODCROFT,2598,WOODCROFT,0.230570,0.170337,0.102332,0.032383,0.464378,WOODCROFT,0.283898,0.117797,0.044915,0.009322,0.544068
254,6680,Satoo,53.446373,-113.458892,SATOO,3391,SATOO,0.266667,0.214425,0.085770,0.022222,0.410916,SATOO,0.190204,0.267755,0.088980,0.019592,0.433469


In [34]:
# remove neighbouthood columns 
edm_hood_total.rename(columns = {'Neighbourhood_x': 'Neighbourhood Name'}, inplace = True)
edm_hood_total.drop(['Neighbourhood_y','Neighbourhood Name_x','Neighbourhood Name_y'],axis = 1,inplace = True)

In [35]:
edm_hood_total

,Neighbourhood Number,Neighbourhood Name,Latitude,Longitude,Population,High School Diploma or below,"Trades, College Certificate or Diploma",University Degree,"Medical degree, Master's or Doctorate",No Response education,"Less than $60,000","$60,000 to $125,000","$125,000 to $200,000","$200,000 or more",No Response income
0,4487,Secord,53.533937,-113.701353,914,0.332569,0.209174,0.150917,0.037615,0.269725,0.323894,0.198230,0.131858,0.030973,0.315044
1,4620,Westview Village,53.552222,-113.696668,2242,0.638655,0.307773,0.025210,0.004202,0.024160,0.197507,0.124640,0.021093,0.000959,0.655801
2,5462,Chappelle Area,53.402917,-113.586845,151,0.054283,0.086359,0.082129,0.027141,0.750088,0.074909,0.263094,0.129111,0.033496,0.499391
3,6216,The Orchards At Ellerslie,53.403087,-113.458166,0,0.065125,0.105904,0.110164,0.026172,0.692635,0.078249,0.327586,0.169761,0.043767,0.380637
4,2670,Rural North East Horse Hill,53.678364,-113.312512,570,0.048499,0.013857,0.032333,0.004619,0.900693,0.056338,0.028169,0.021127,0.028169,0.866197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,3170,Dovercourt,53.575727,-113.559534,2048,0.409021,0.337920,0.152141,0.042049,0.058869,0.255196,0.284065,0.107390,0.024249,0.329099
252,5300,Ogilvie Ridge,53.464904,-113.567069,1001,0.146628,0.170088,0.222874,0.151026,0.309384,0.038889,0.108333,0.086111,0.175000,0.591667
253,3450,Woodcroft,53.564595,-113.558327,2598,0.230570,0.170337,0.102332,0.032383,0.464378,0.283898,0.117797,0.044915,0.009322,0.544068
254,6680,Satoo,53.446373,-113.458892,3391,0.266667,0.214425,0.085770,0.022222,0.410916,0.190204,0.267755,0.088980,0.019592,0.433469


In [311]:
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="edm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of edmonton are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of edmonton are 53.535411, -113.507996.


In [314]:
# create map of Edmonton
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(edm_hood_total['Latitude'], edm_hood_total['Longitude'], edm_hood_total['Neighbourhood Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edmonton)  
    
map_edmonton

## Getting the second set of data 
### the second set of data will be popular locations are within a neighbourhood and how often they are visited 

In [ ]:
#foursquare keys 
CLIENT_ID = '1J22M2N15T0T1SX1RID4X3ARVEUE0SLBCTCJOS3L1M24RVVW'
CLIENT_SECRET = 'VJAOZHIQOETDQJ0NTU1BOFOVRWUBWFSDHXFDQUCKZWFK4L1X' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Exploriong Neighborhoods 
### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    CLIENT_ID = ''
    CLIENT_SECRET = '' 
    VERSION = '20180605'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)